In [1]:
import polars as pl
from datetime import date

In [2]:
schema_pays = {
    "pay_date": pl.Date,
    "total": pl.Float64,
    "user_id": pl.Int64,
    "value_prop": pl.Utf8,
}

df_pays = pl.read_csv("../data/raw/pays.csv", schema=schema_pays)

df_pays.filter(pl.col("user_id") == 50807).head(10)

pay_date,total,user_id,value_prop
date,f64,i64,str
2020-11-09,70.98,50807,"""credits_consumer"""
2020-11-11,16.09,50807,"""credits_consumer"""
2020-11-11,107.31,50807,"""send_money"""
2020-11-15,37.53,50807,"""transport"""
2020-11-15,52.65,50807,"""cellphone_recharge"""
2020-11-26,21.96,50807,"""cellphone_recharge"""


In [3]:
schema_taps = {
    "day": pl.Date,
    "event_data": pl.Struct([
        pl.Field("position", pl.Int64),
        pl.Field("value_prop", pl.Utf8),
    ]),
    "user_id": pl.Int64,
}

df_taps = pl.read_ndjson("../data/raw/taps.json", schema=schema_taps)
df_taps = df_taps.unnest("event_data")

df_taps

day,position,value_prop,user_id
date,i64,str,i64
2020-11-01,0,"""cellphone_recharge""",98702
2020-11-01,2,"""point""",3708
2020-11-01,3,"""send_money""",3708
2020-11-01,0,"""transport""",93963
2020-11-01,1,"""cellphone_recharge""",93963
…,…,…,…
2020-11-30,2,"""credits_consumer""",64149
2020-11-30,1,"""point""",3329
2020-11-30,1,"""cellphone_recharge""",77012


In [4]:
schema_prints = {
    "day": pl.Date,
    "event_data": pl.Struct([
        pl.Field("position", pl.Int64),
        pl.Field("value_prop", pl.Utf8),
    ]),
    "user_id": pl.Int64,
}

df_prints = pl.read_ndjson("../data/raw/prints.json", schema=schema_prints)
df_prints = df_prints.unnest("event_data")

df_prints

day,position,value_prop,user_id
date,i64,str,i64
2020-11-01,0,"""cellphone_recharge""",98702
2020-11-01,1,"""prepaid""",98702
2020-11-01,0,"""prepaid""",63252
2020-11-01,0,"""cellphone_recharge""",24728
2020-11-01,1,"""link_cobro""",24728
…,…,…,…
2020-11-30,0,"""send_money""",50807
2020-11-30,1,"""prepaid""",50807
2020-11-30,2,"""credits_consumer""",50807


In [5]:
def get_last_week(df, column_name):
    return (
    df
    .with_columns([
        pl.col(column_name).dt.truncate("1w").alias("week_start")
    ])
    .filter(pl.col("week_start") == pl.col("week_start").max())
    .drop("week_start")
)


def get_last_weeks(df, column_name):
    return (
        df
        .with_columns(pl.col(column_name).dt.truncate("1w").alias("week_start"))
        .filter(
            pl.col("week_start").is_in(
                pl.col("week_start").unique().sort().tail(4).head(3).implode()
            )
        )
        .drop("week_start")
    )

In [6]:
df_final = get_last_week(df_prints, "day")

df_final.filter(pl.col("user_id") == 50807).head(10)

day,position,value_prop,user_id
date,i64,str,i64
2020-11-30,0,"""send_money""",50807
2020-11-30,1,"""prepaid""",50807
2020-11-30,2,"""credits_consumer""",50807


In [7]:
## Cantidad de veces que vio

counts = (
    get_last_weeks(df_prints, "day")
    .group_by(["user_id", "value_prop"])
    .agg(pl.len().cast(pl.Int64).alias("cantidad_vistas"))
)

out = df_final.join(
    counts,
    on=["user_id", "value_prop"],
    how="left",
).with_columns(pl.col("cantidad_vistas").fill_null(0))

out.filter(pl.col("user_id") == 50807).head(10)

day,position,value_prop,user_id,cantidad_vistas
date,i64,str,i64,i64
2020-11-30,0,"""send_money""",50807,0
2020-11-30,1,"""prepaid""",50807,1
2020-11-30,2,"""credits_consumer""",50807,2


In [8]:
## Cantidad de veces que clickeo y si hizo click

counts = (
    get_last_weeks(df_taps, "day")
    .group_by(["user_id", "value_prop"])
    .agg(pl.len().cast(pl.Int64).alias("cantidad_taps"))
)


out = (
    out
    .join(counts, on=["user_id", "value_prop"], how="left")
    .with_columns(
        pl.col("cantidad_taps").fill_null(0).cast(pl.Int64).alias("cantidad_taps")
    )
    .with_columns(
        (pl.col("cantidad_taps") > 0).alias("hizo_click")
    )
)

out.filter(pl.col("user_id") == 50807).head(10)

day,position,value_prop,user_id,cantidad_vistas,cantidad_taps,hizo_click
date,i64,str,i64,i64,i64,bool
2020-11-30,0,"""send_money""",50807,0,0,false
2020-11-30,1,"""prepaid""",50807,1,0,false
2020-11-30,2,"""credits_consumer""",50807,2,0,false


In [9]:
## Cantidad de veces que pago

counts = (
    get_last_weeks(df_pays, "pay_date")
    .group_by(["user_id", "value_prop"])
    .agg(pl.len().cast(pl.Int64).alias("cantidad_pagos"))
)

out = out.join(
    counts,
    on=["user_id", "value_prop"],
    how="left",
).with_columns(pl.col("cantidad_pagos").fill_null(0))

out.filter(pl.col("user_id") == 50807).head(10)

day,position,value_prop,user_id,cantidad_vistas,cantidad_taps,hizo_click,cantidad_pagos
date,i64,str,i64,i64,i64,bool,i64
2020-11-30,0,"""send_money""",50807,0,0,false,1
2020-11-30,1,"""prepaid""",50807,1,0,false,0
2020-11-30,2,"""credits_consumer""",50807,2,0,false,2


In [10]:
## Cantidad de veces que pago

counts = (
    get_last_weeks(df_pays, "pay_date")
    .group_by(["user_id", "value_prop"])
    .agg(pl.sum("total").cast(pl.Int64).alias("total_pagos"))
)

out = out.join(
    counts,
    on=["user_id", "value_prop"],
    how="left",
).with_columns(pl.col("total_pagos").fill_null(0))

out.filter(pl.col("user_id") == 50807).head(10)

day,position,value_prop,user_id,cantidad_vistas,cantidad_taps,hizo_click,cantidad_pagos,total_pagos
date,i64,str,i64,i64,i64,bool,i64,i64
2020-11-30,0,"""send_money""",50807,0,0,false,1,107
2020-11-30,1,"""prepaid""",50807,1,0,false,0,0
2020-11-30,2,"""credits_consumer""",50807,2,0,false,2,87


In [11]:
today = date.today()

out.write_csv(f"../data/out/{today}.csv")
out.write_parquet(f"../data/out/{today}.parquet")